<a href="https://colab.research.google.com/github/Medalcode/Colab1/blob/main/Eval3Mineria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Avance para Retroalimentación Evaluacion 3 Mineria de Datos**

# **Integrantes: Camila Troncoso - Jonatthan Medalla**

In [15]:
import pandas as pd # Importa la librería pandas para manipulación de datos (DataFrames).
import numpy as np # Importa la librería numpy para operaciones numéricas.
import io # Módulo para trabajar con streams de I/O.
import base64 # Módulo para codificación y decodificación Base64.
import matplotlib.pyplot as plt # Importa matplotlib para la creación de gráficos.
import seaborn as sns # Importa seaborn para visualizaciones estadísticas, basado en matplotlib.
import gradio as gr # Importa Gradio para construir interfaces de usuario interactivas.
from sklearn.preprocessing import MinMaxScaler, StandardScaler # Importa escaladores para normalización y estandarización.
from scipy.stats import skew, kurtosis # Importa funciones para calcular asimetría y curtosis.
import os # Módulo para interactuar con el sistema operativo (e.g., rutas de archivos).
from io import BytesIO # Clase para manejar streams de bytes en memoria.

# Install missing dependency
!pip install rfc3987 # Instala la librería rfc3987, necesaria para algunas funcionalidades de Gradio o sus dependencias.

In [16]:
# Configuración inicial para Matplotlib/Seaborn
sns.set_theme(style="whitegrid") # Establece un tema visual para Seaborn con un fondo de cuadrícula blanco.

In [17]:
# Variables globales para almacenar el DataFrame y el historial de operaciones (Log)
estado_df = None # Variable global para mantener el DataFrame actual procesado.
entradas_log = [] # Lista global para registrar las operaciones realizadas en el DataFrame.

In [18]:
def get_columnas_numericas(df_entrada):
    """Función auxiliar para obtener columnas numéricas de un DataFrame."""
    if df_entrada is not None:
        # Selecciona columnas con tipos de datos numéricos y retorna sus nombres como una lista.
        return df_entrada.select_dtypes(include=np.number).columns.tolist()
    return [] # Retorna una lista vacía si el DataFrame es None.

In [19]:
def cargar_datos(archivo_obj, delimitador_elegido):
    """I.1 cargar_datos: Carga el archivo subido en un DataFrame de pandas."""
    global estado_df, entradas_log # Accede a las variables globales estado_df y entradas_log.
    entradas_log = [] # Reinicia el log en cada nueva carga para mantener un historial limpio de la sesión actual.

    if archivo_obj is None:
        return None, "Error: Debe subir un archivo.", None # Si no hay archivo, retorna un error.

    ruta_archivo = archivo_obj.name # Obtiene la ruta física del archivo subido.

    try:
        # Procesa archivos CSV
        if ruta_archivo.endswith('.csv'):
            # Determina el delimitador basado en la selección del usuario.
            if delimitador_elegido == "Coma (,)":
                delimitador = ','
            elif delimitador_elegido == "Punto y Coma (;)":
                delimitador = ';'
            else:
                delimitador = ',' # Delimitador por defecto si no se especifica o es inválido.

            # Intentar leer el archivo con codificaciones comunes para manejar diferentes formatos.
            try:
                df_datos = pd.read_csv(ruta_archivo, delimiter=delimitador, encoding='utf-8')
            except UnicodeDecodeError:
                df_datos = pd.read_csv(ruta_archivo, delimiter=delimitador, encoding='ISO-8859-1')

            # ***** NUEVA LÓGICA: Detección de CSV de una sola columna *****
            if df_datos.shape[1] == 1: # Verifica si el DataFrame tiene solo una columna.
                estado_df = None # Reinicia el DataFrame de estado.
                # Retorna un error si el CSV tiene una sola columna, sugiriendo un delimitador incorrecto.
                return None, "Error: El archivo CSV cargado tiene una sola columna. Asegúrese de seleccionar el delimitador correcto o que el archivo contenga múltiples columnas para un análisis significativo.", None
            # ************************************************************

        # Procesa archivos Excel (xls o xlsx)
        elif ruta_archivo.endswith(('.xls', '.xlsx')):
            df_datos = pd.read_excel(ruta_archivo) # Lee el archivo Excel.

        # Maneja tipos de archivo no soportados.
        else:
            return None, "Error: Archivo no válido. Suba un archivo CSV o Excel.", None # Retorna un error para tipos de archivo no soportados.

        # Validación de tipos de datos: verifica que el DataFrame contenga columnas categóricas y numéricas.
        columnas_numericas = df_datos.select_dtypes(include=np.number).columns
        columnas_categoricas = df_datos.select_dtypes(include=['object', 'category']).columns

        # Genera un mensaje de advertencia si faltan tipos de datos importantes para el análisis.
        if len(columnas_numericas) == 0 or len(columnas_categoricas) == 0:
            mensaje = "Advertencia: El archivo debe contener tanto datos categóricos como numéricos para un análisis completo."
        else:
            mensaje = f"Archivo cargado. Dimensiones: {df_datos.shape}. {len(columnas_numericas)} numéricas, {len(columnas_categoricas)} categóricas."

        estado_df = df_datos # Almacena el DataFrame cargado en la variable de estado global.
        # Registra la operación de carga en el log.
        entradas_log.append(f"Carga: Archivo {os.path.basename(ruta_archivo)} cargado. Se detectaron {df_datos.shape[0]} filas.")

        # Retorna el DataFrame, el mensaje y una vista previa de las primeras filas.
        return estado_df, mensaje, gr.Dataframe(value=df_datos.head())

    except Exception as e:
        estado_df = None # Reinicia el DataFrame de estado si ocurre un error.
        return None, f"Error de lectura: {str(e)}", None # Retorna un mensaje de error detallado.

In [20]:
def manejar_valores_nulos(df_entrada, nombres_columnas_str, metodo):
    """II.1 manejar_valores_nulos: Manejo interactivo de valores nulos utilizando diferentes estrategias."""
    global estado_df, entradas_log # Accede a las variables globales.
    # Crea una copia del DataFrame de entrada para no modificar el original directamente hasta que la operación se confirme.
    df_procesado = df_entrada.copy() if df_entrada is not None else None

    # Verifica si hay un DataFrame cargado; si no, retorna un error.
    if df_procesado is None:
        return None, "Error: Primero cargue un archivo.", None

    # Procesa la cadena de nombres de columnas para obtener una lista, eliminando espacios en blanco.
    nombres_columnas = [c.strip() for c in nombres_columnas_str.split(",") if c.strip()]
    if not nombres_columnas:
        # Si el usuario no especificó columnas, aplicar a todas las columnas numéricas automáticamente.
        nombres_columnas = get_columnas_numericas(df_procesado)
        if not nombres_columnas:
            return estado_df, "Advertencia: No hay columnas numéricas para aplicar la limpieza de nulos.", gr.Dataframe(value=estado_df.head())

    # Calcula el número total de valores nulos antes del tratamiento en las columnas seleccionadas.
    total_nulos_previos = df_procesado[nombres_columnas].isnull().sum().sum()
    filas_afectadas = 0 # Inicializa el contador de filas o valores afectados.

    # Si no hay nulos en las columnas seleccionadas, no se realiza ninguna operación.
    if total_nulos_previos == 0:
        mensaje = "No se encontraron valores nulos en las columnas seleccionadas. No se realizó ninguna operación."
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    # Aplica el método de "Eliminar filas" si es seleccionado.
    if metodo == "Eliminar filas":
        total_filas_originales = len(df_procesado)
        df_procesado = df_procesado.dropna(subset=nombres_columnas) # Elimina filas con nulos en las columnas especificadas.
        filas_afectadas = total_filas_originales - len(df_procesado) # Calcula cuántas filas fueron eliminadas.
        # Registra la operación en el log.
        entradas_log.append(f"Limpieza Nulos: Se eliminaron {filas_afectadas} filas con nulos en {', '.join(nombres_columnas)}.")

    else:
        # Itera sobre cada columna para aplicar los métodos de imputación.
        for col in nombres_columnas:
            # Verifica si la columna existe y es numérica antes de imputar.
            if col not in df_procesado.columns or col not in get_columnas_numericas(df_procesado):
                entradas_log.append(f"Advertencia: La columna '{col}' no es numérica o no existe para imputación. Se omitió.")
                continue

            valor_imputacion = 0 # Valor por defecto para evitar errores si no se selecciona un método válido.
            # Determina el valor de imputación según el método seleccionado.
            if metodo == "Llenar con promedio":
                valor_imputacion = df_procesado[col].mean() # Imputa con la media de la columna.
            elif metodo == "Llenar con máximo":
                valor_imputacion = df_procesado[col].max() # Imputa con el valor máximo de la columna.
            elif metodo == "Llenar con mínimo":
                valor_imputacion = df_procesado[col].min() # Imputa con el valor mínimo de la columna.
            elif metodo == "Llenar con cero":
                valor_imputacion = 0 # Imputa con cero.

            nulos_en_columna = df_procesado[col].isnull().sum() # Cuenta los nulos en la columna actual.
            df_procesado[col] = df_procesado[col].fillna(valor_imputacion) # Rellena los nulos con el valor calculado.
            filas_afectadas += nulos_en_columna # Suma los nulos que realmente se llenaron.
            # Registra la operación en el log.
            entradas_log.append(f"Limpieza Nulos: La columna '{col}' se imputó con {metodo.split(' ')[-1]} ({valor_imputacion:.2f}).")

    # Actualiza el DataFrame de estado global con el DataFrame procesado.
    estado_df = df_procesado
    # Prepara el mensaje de éxito para el usuario.
    mensaje = f"Limpieza completada. {total_nulos_previos} valores nulos tratados. Registros afectados: {filas_afectadas}."
    # Retorna el DataFrame actualizado, el mensaje y una vista previa.
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [21]:
def aplicar_escalado(df_entrada, nombres_columnas_str, metodo_escalado):
    """II.2 aplicar_escalado: Aplica normalización (Min-Max) o estandarización (Z-Score) a columnas numéricas."""
    global estado_df, entradas_log # Accede a las variables globales.
    # Crea una copia del DataFrame de entrada para evitar modificar el original directamente.
    df_escalado = df_entrada.copy() if df_entrada is not None else None

    # Verifica si hay un DataFrame cargado; si no, retorna un error.
    if df_escalado is None:
        return None, "Error: Primero cargue un archivo.", None

    # Procesa la cadena de nombres de columnas para obtener una lista de columnas a escalar.
    nombres_columnas = [c.strip() for c in nombres_columnas_str.split(",") if c.strip()]
    # Si no se especifican columnas, se retorna un error.
    if not nombres_columnas:
        return estado_df, "Error: Debe especificar al menos una columna numérica para escalar.", gr.Dataframe(value=estado_df.head())

    # Valida que todas las columnas especificadas existan y sean numéricas.
    if not all(col in df_escalado.columns and col in get_columnas_numericas(df_escalado) for col in nombres_columnas):
        return estado_df, "Error: Verifique que las columnas existan y sean numéricas.", gr.Dataframe(value=estado_df.head())

    escalador = None # Inicializa la variable del escalador.
    justificacion_escalado = "" # Inicializa la justificación del escalado.
    # Selecciona el escalador y la justificación según el método elegido.
    if metodo_escalado == "Min-Max":
        escalador = MinMaxScaler() # Utiliza MinMaxScaler para normalización (rango [0, 1]).
        justificacion_escalado = "**Recomendación Min-Max:** Se recomienda para algoritmos que esperan un rango acotado (ej. Redes Neuronales) o cuando la distribución no es gaussiana. Sin embargo, es sensible a los *outliers* [10-12]."
    elif metodo_escalado == "Z-Score":
        escalador = StandardScaler() # Utiliza StandardScaler para estandarización (media 0, desviación estándar 1).
        justificacion_escalado = "**Recomendación Z-Score:** Se recomienda para algoritmos basados en distancias (ej. K-Means, KNN) o cuando se asume una distribución aproximadamente normal. Es menos sensible a los *outliers* que Min-Max [10, 12, 13]."
    else:
        return estado_df, "Método de escalado no válido.", gr.Dataframe(value=estado_df.head())

    # Aplica el escalado a cada columna seleccionada.
    for col in nombres_columnas:
        # Transforma la columna usando el escalador elegido. Se usa [[col]] para mantener la dimensión 2D requerida por fit_transform.
        df_escalado[col] = escalador.fit_transform(df_escalado[[col]])
        entradas_log.append(f"Escalado: Columna '{col}' escalada usando {metodo_escalado}.") # Registra la operación en el log.

    estado_df = df_escalado # Actualiza el DataFrame de estado global con el DataFrame escalado.
    # Prepara el mensaje de éxito para el usuario.
    mensaje = f"Escalado de {', '.join(nombres_columnas)} completado usando {metodo_escalado}. {justificacion_escalado}"
    # Retorna el DataFrame actualizado, el mensaje y una vista previa.
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [22]:
def detectar_y_tratar_outliers(df_entrada, nombre_columna, tratamiento):
    """II.3 detectar_y_tratar_outliers: Detección de valores atípicos (outliers) por el método IQR y aplicación de tratamientos."""
    global estado_df, entradas_log # Accede a las variables globales.
    # Crea una copia del DataFrame de entrada para evitar modificar el original directamente.
    df_outliers_tratado = df_entrada.copy() if df_entrada is not None else None

    # Verifica si hay un DataFrame cargado; si no, retorna un error.
    if df_outliers_tratado is None:
        return None, "Error: Primero cargue un archivo.", None

    # Valida que la columna especificada exista y sea numérica.
    if nombre_columna not in df_outliers_tratado.columns or nombre_columna not in get_columnas_numericas(df_outliers_tratado):
        return estado_df, f"Error: La columna '{nombre_columna}' no existe o no es numérica.", gr.Dataframe(value=estado_df.head())

    # Calcula los cuartiles (Q1 y Q3) de la columna para el método IQR (Rango Intercuartílico).
    cuartil_1 = df_outliers_tratado[nombre_columna].quantile(0.25)
    cuartil_3 = df_outliers_tratado[nombre_columna].quantile(0.75)
    # Calcula el Rango Intercuartílico (IQR).
    rango_iqr = cuartil_3 - cuartil_1
    # Define los límites inferior y superior para la detección de outliers (1.5 * IQR).
    limite_inferior_iqr = cuartil_1 - 1.5 * rango_iqr
    limite_superior_iqr = cuartil_3 + 1.5 * rango_iqr

    # Identifica los registros que son outliers (fuera de los límites IQR).
    registros_outliers = df_outliers_tratado[(df_outliers_tratado[nombre_columna] < limite_inferior_iqr) | (df_outliers_tratado[nombre_columna] > limite_superior_iqr)]
    numero_outliers = len(registros_outliers)

    # Si no se detectan outliers, informa al usuario y no realiza ninguna acción.
    if numero_outliers == 0:
        mensaje = f"No se detectaron *outliers* en la columna '{nombre_columna}' (Método IQR)."
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    # Aplica el tratamiento seleccionado por el usuario.
    if tratamiento == "Eliminar registros":
        # Filtra el DataFrame para eliminar las filas que contienen outliers.
        df_outliers_tratado = df_outliers_tratado[~((df_outliers_tratado[nombre_columna] < limite_inferior_iqr) | (df_outliers_tratado[nombre_columna] > limite_superior_iqr))]
        # Registra la operación en el log.
        entradas_log.append(f"Outliers: Se eliminaron {numero_outliers} *outliers* en '{nombre_columna}'.")
        mensaje = f"Se detectaron y eliminaron {numero_outliers} *outliers* en '{nombre_columna}'. Se eliminaron {numero_outliers} filas."

    elif tratamiento == "Capping (Winsorización)":
        # Limita los valores atípicos a los umbrales IQR (Winsorización).
        df_outliers_tratado[nombre_columna] = np.where(df_outliers_tratado[nombre_columna] > limite_superior_iqr, limite_superior_iqr, df_outliers_tratado[nombre_columna])
        df_outliers_tratado[nombre_columna] = np.where(df_outliers_tratado[nombre_columna] < limite_inferior_iqr, limite_inferior_iqr, df_outliers_tratado[nombre_columna])
        # Registra la operación en el log.
        entradas_log.append(f"Outliers: Se aplicó *capping* a {numero_outliers} *outliers* en '{nombre_columna}'.")
        mensaje = f"Se detectaron {numero_outliers} *outliers* y se aplicó *Capping* (Winsorización) para conservar los registros."

    else: # Opción "Informar" (no modifica el DataFrame)
        # Simplemente informa sobre la presencia de outliers sin modificarlos.
        mensaje = f"Se detectaron {numero_outliers} *outliers* en '{nombre_columna}'. Se recomienda tratarlos, ya que pueden sesgar la media y la desviación estándar [15, 21]."
        estado_df = df_entrada # No se modifica el DataFrame si solo se informa.
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    # Actualiza el DataFrame de estado global con el DataFrame procesado.
    estado_df = df_outliers_tratado
    # Retorna el DataFrame actualizado, el mensaje y una vista previa.
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [23]:
def ejecutar_analisis(df_entrada):
    """III.1 ejecutar_analisis: Calcula estadísticas descriptivas, correlación, curtosis y asimetría de las columnas numéricas."""
    global entradas_log # Accede a la variable global de registro.
    # Verifica si hay un DataFrame cargado; si no, retorna un error.
    if df_entrada is None:
        return "Error: Primero cargue y procese el archivo.", None

    # Selecciona solo las columnas numéricas del DataFrame para el análisis.
    df_numerico = df_entrada.select_dtypes(include=np.number)

    # Verifica si existen columnas numéricas en el DataFrame.
    if df_numerico.empty:
        return "El DataFrame no contiene columnas numéricas para el análisis estadístico.", None

    # Calcula las estadísticas descriptivas (media, desviación estándar, cuartiles, etc.).
    estadisticas_descriptivas = df_numerico.describe().T
    # Calcula la matriz de correlación de Pearson entre las columnas numéricas.
    matriz_correlacion = df_numerico.corr(method='pearson')

    # Calcula la curtosis de cada columna numérica (Fisher=False para que la normal sea 3).
    series_curtosis = df_numerico.apply(kurtosis, fisher=False)
    # Calcula la asimetría (skewness) de cada columna numérica.
    series_asimetria = df_numerico.apply(skew)

    # Combina los resultados de curtosis y asimetría en un solo DataFrame.
    df_forma_distribucion = pd.DataFrame({
        'Curtosis (Normal ≈ 3)': series_curtosis,
        'Asimetría (Skewness)': series_asimetria
    }).round(3)

    # Prepara el texto de interpretación de los resultados.
    texto_interpretacion = "### Resumen de Interpretación:\n"
    texto_interpretacion += "- **Curtosis:** Los valores > 3 (Leptocúrtica) indican un pico más agudo y colas pesadas, sugiriendo más *outliers* [30].\n"
    texto_interpretacion += "- **Asimetría:** Valores positivos (> 0) indican sesgo a la derecha (media > mediana) [29, 31].\n"
    texto_interpretacion += "- **Correlación:** Los valores cercanos a 1 o -1 en el mapa de calor indican relaciones lineales fuertes entre pares de variables [32].\n"

    # Registra la operación de análisis en el log.
    entradas_log.append("Análisis Estadístico: Cálculos descriptivos, curtosis y asimetría generados.")

    # Formatea el resumen del análisis para mostrarlo al usuario.
    resumen_analisis_texto = (
        f"{texto_interpretacion}\n\n"
        f"**Estadísticas Descriptivas (Media, Desviación, Cuartiles):**\n{estadisticas_descriptivas.to_markdown()}\n\n"
        f"**Forma de la Distribución (Curtosis y Asimetría):**\n{df_forma_distribucion.to_markdown()}\n"
    )

    # Retorna el resumen en texto y la matriz de correlación (esta última para posible uso interno o visualización).
    return resumen_analisis_texto, matriz_correlacion

In [24]:
def generar_graficos(df_entrada, columna_correlacion_heatmap, columna_distribucion_plot):
    """III.2 generar_graficos: Genera un mapa de calor de correlaciones y un histograma/boxplot para una columna seleccionada."""
    global entradas_log # Accede a la variable global de registro.

    # Verifica si hay un DataFrame cargado; si no, retorna un error.
    if df_entrada is None:
        return None, None, "Error: Primero cargue el archivo."

    # Selecciona solo las columnas numéricas del DataFrame para la generación de gráficos.
    df_numerico = df_entrada.select_dtypes(include=np.number)

    # Si no hay columnas numéricas, advierte al usuario.
    if df_numerico.empty:
        return None, None, "Advertencia: No hay columnas numéricas para generar gráficos."

    ruta_plot_correlacion = None
    ruta_plot_distribucion = None

    try:
        # Crea una figura para el mapa de calor de correlaciones.
        plt.figure(figsize=(10, 8))
        # Genera el mapa de calor usando Seaborn, mostrando los valores de correlación y una paleta de color.
        sns.heatmap(df_numerico.corr(), annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Mapa de Calor de Correlaciones (Pearson)")
        # Guarda el mapa de calor como una imagen PNG.
        ruta_plot_correlacion = "correlation_plot.png"
        plt.savefig(ruta_plot_correlacion)
        plt.close() # Cierra la figura para liberar memoria.
        entradas_log.append("Visualización: Mapa de calor de correlaciones generado.") # Registra la operación.
    except Exception as e:
        entradas_log.append(f"Error al generar mapa de correlación: {e}") # Registra cualquier error que ocurra.

    # Verifica si se ha especificado una columna para el gráfico de distribución y si esta es numérica.
    if columna_distribucion_plot and columna_distribucion_plot in df_numerico.columns:
        try:
            # Crea una figura con dos subplots para el histograma y el boxplot.
            figura, ejes = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

            # Genera un histograma con la función de densidad de kernel (KDE) para mostrar la distribución.
            sns.histplot(df_numerico[columna_distribucion_plot], kde=True, ax=ejes[0])
            ejes[0].set_title(f"Distribución de: {columna_distribucion_plot} (Histograma y KDE)")

            # Genera un boxplot para visualizar la distribución, la mediana y los posibles outliers (valores atípicos).
            sns.boxplot(x=df_numerico[columna_distribucion_plot], ax=ejes[1])
            ejes[1].set_title(f"Boxplot de: {columna_distribucion_plot} (Outliers: 1.5*IQR)")

            plt.tight_layout() # Ajusta el diseño para evitar superposiciones entre subplots.
            # Guarda los gráficos de distribución como una imagen PNG.
            ruta_plot_distribucion = "distribution_plot.png"
            plt.savefig(ruta_plot_distribucion)
            plt.close() # Cierra la figura para liberar memoria.
            entradas_log.append(f"Visualización: Gráfico de distribución para '{columna_distribucion_plot}' generado.") # Registra la operación.
        except Exception as e:
            entradas_log.append(f"Error al generar gráfico de distribución para '{columna_distribucion_plot}': {e}") # Registra cualquier error.
    else:
        entradas_log.append("Advertencia: No se pudo generar el gráfico de distribución, columna no numérica o inexistente.")

    # Retorna las rutas de los archivos generados y un mensaje de estado.
    return ruta_plot_correlacion, ruta_plot_distribucion, "Gráficos generados correctamente." if ruta_plot_correlacion or ruta_plot_distribucion else "No se pudo generar ningón gráfico."


In [25]:
def exportar_resultados(df_entrada, formato_exportacion):
    """IV.1 exportar_resultados: Permite la exportación del DataFrame procesado y genera un reporte de log de las operaciones."""
    global entradas_log # Accede a la variable global de registro.

    # Verifica si hay un DataFrame procesado; si no, retorna un error.
    if df_entrada is None:
        return "Error: No hay datos procesados para exportar.", None, None

    ruta_reporte = "reporte_analisis.txt" # Define el nombre del archivo del reporte de log.
    contenido_log = "\n".join(entradas_log) # Concatena todas las entradas del log en una cadena, separadas por saltos de línea.

    # Prepara el contenido final del reporte incluyendo un resumen automático y la interpretación.
    contenido_reporte_final = (
        "### REPORTE BREVE AUTOMÁTICO DE PROCESAMIENTO DE DATOS\n\n"
        "**Proceso Seguido y Decisiones Tomadas en Limpieza de Datos:**\n"
        f"{contenido_log}\n\n"
        f"**Interpretación Preliminar de Resultados Obtenidos:**\n"
        f"(La interpretación completa de correlaciones, curtosis y regresiones debe realizarla el analista.)\n"
        f"Se recomienda revisar el *heatmap* para correlaciones fuertes (Pearson > 0.7 o < -0.7) [32, 38].\n"
        f"La limpieza de datos asegura la calidad y reduce el sesgo en fases de modelado posteriores (GIGO: *Garbage In, Garbage Out*) [39].\n"
        f"Dimensiones del DataFrame final: {df_entrada.shape}\n"
    )

    # Escribe el contenido del reporte en un archivo de texto.
    with open(ruta_reporte, "w") as f:
        f.write(contenido_reporte_final)

    ruta_salida = None # Inicializa la ruta del archivo de datos procesados.
    # Exporta el DataFrame procesado según el formato seleccionado por el usuario.
    if formato_exportacion == "CSV":
        ruta_salida = "datos_procesados.csv"
        df_entrada.to_csv(ruta_salida, index=False) # Exporta a CSV sin el índice del DataFrame.
    elif formato_exportacion == "Excel":
        ruta_salida = "datos_procesados.xlsx"
        df_entrada.to_excel(ruta_salida, index=False) # Exporta a Excel sin el índice del DataFrame.
    else:
        return "Error: Formato de exportación no válido.", None, None # Retorna un error si el formato no es válido.

    # Registra la operación de exportación en el log.
    entradas_log.append(f"Exportación: Datos procesados guardados en {ruta_salida} y Log generado.")

    # Retorna un mensaje de éxito, la ruta del archivo de datos y la ruta del reporte.
    return f"Exportación exitosa. Descargue el archivo y el reporte.", ruta_salida, ruta_reporte

In [26]:
def update_dropdown_choices(df_entrada):
    """Actualiza las opciones del dropdown de columnas numéricas en la interfaz de Gradio, útil después de operaciones que pueden cambiar las columnas."""
    if df_entrada is not None:
        # Asume que get_columnas_numericas ya está definida en otra celda y accesible.
        numeric_cols = get_columnas_numericas(df_entrada) # Obtiene la lista de columnas numéricas del DataFrame actual.
        # Retorna un componente Dropdown actualizado con las columnas numéricas como opciones.
        # Si hay columnas, selecciona la primera por defecto; de lo contrario, deja el valor en None.
        return gr.Dropdown(choices=numeric_cols, value=numeric_cols[0] if numeric_cols else None)
    # Si no hay DataFrame, retorna un Dropdown vacío.
    return gr.Dropdown(choices=[], value=None)

In [27]:
import gradio as gr # Importa la librería Gradio para construir la interfaz de usuario.

# Define la interfaz de Gradio como un bloque, con un título principal.
with gr.Blocks(title="Aplicación de Minería de Datos y EDA") as interfaz:
    gr.Markdown("## \U0001f528️ Aplicación Interactiva para Procesamiento y Análisis de Datos") # Título principal de la aplicación.

    estado_df_gradio = gr.State(None) # Un componente de estado de Gradio para mantener el DataFrame entre interacciones, sin mostrarlo directamente.

    # Define la primera pestaña para la carga de datos.
    with gr.Tab("1. Carga de Datos"):
        gr.Markdown("### Carga y Validación del Archivo") # Subtítulo para la sección de carga.
        with gr.Row(): # Organiza los componentes en una fila.
            # Componente para subir el archivo (CSV o Excel).
            input_archivo = gr.File(label="Subir Archivo (CSV o Excel)", interactive=True)
            # Radio buttons para seleccionar el delimitador del archivo CSV.
            radio_separador = gr.Radio(
                choices=["Coma (,)", "Punto y Coma (;)"],
                label="Selecciona el Separador del Archivo",
                value="Coma (,)",
                interactive=True
            )

        btn_cargar_datos = gr.Button("Cargar y Validar") # Botón para iniciar la carga de datos.
        msg_carga_datos = gr.Textbox(label="Mensaje de Carga") # Muestra mensajes de estado de la carga.
        df_vista_previa = gr.Dataframe(label="Vista Previa (5 primeras filas)") # Muestra las primeras filas del DataFrame cargado.

    # Define la segunda pestaña para el procesamiento y limpieza de datos.
    with gr.Tab("2. Procesamiento y Limpieza (Preparación de Datos)"):
        gr.Markdown("### Limpieza de Valores Nulos") # Subtítulo para la sección de nulos.
        with gr.Row(): # Organiza los componentes en una fila.
            radio_metodo_nulos = gr.Radio(
                choices=["Eliminar filas", "Llenar con promedio", "Llenar con máximo", "Llenar con mínimo", "Llenar con cero"],
                label="Método para manejar nulos [6, 41]",
                value="Eliminar filas"
            )
            input_col_nulos = gr.Textbox(label="Columnas para Limpieza (Separadas por comas)", placeholder="Ej: Col1, Col2 (Dejar vacío para todo el DF)")

        btn_aplicar_nulos = gr.Button("Aplicar Limpieza de Nulos") # Botón para aplicar la limpieza de nulos.
        msg_resultado_nulos = gr.Textbox(label="Resultado Nulos") # Muestra el resultado de la operación de nulos.

        gr.Markdown("### Normalización y Estandarización") # Subtítulo para la sección de escalado.
        with gr.Row(): # Organiza los componentes en una fila.
            radio_metodo_escalado = gr.Radio(
                choices=["Min-Max", "Z-Score"],
                label="Método de Escalado [10, 42]",
                value="Z-Score"
            )
            input_col_escalar = gr.Textbox(label="Columnas Numéricas para Escalar (Separadas por comas)", placeholder="Ej: Edad, Salario")

        btn_aplicar_escalado = gr.Button("Aplicar Normalización / Estandarización") # Botón para aplicar el escalado.
        msg_resultado_escalado = gr.Textbox(label="Resultado Normalización y Justificación [10]") # Muestra el resultado del escalado.

        gr.Markdown("### Detección y Tratamiento de Outliers (IQR)") # Subtítulo para la sección de outliers.
        with gr.Row(): # Organiza los componentes en una fila.
            input_col_outliers = gr.Textbox(label="Columna para Detección de Outliers (Una sola columna)", placeholder="Ej: Ingresos")
            radio_tratamiento_outliers = gr.Radio(
                choices=["Informar", "Eliminar registros", "Capping (Winsorización)"],
                label="Tratamiento de Outliers [10, 19, 20]",
                value="Informar"
            )

        btn_detectar_outliers = gr.Button("Detectar y Tratar Outliers") # Botón para detectar y tratar outliers.
        msg_resultado_outliers = gr.Textbox(label="Resultado Outliers") # Muestra el resultado del tratamiento de outliers.

    # Define la tercera pestaña para análisis y visualización.
    with gr.Tab("3. Análisis y Visualización"):
        gr.Markdown("### Análisis Estadístico (Correlación, Curtosis y Asimetría) [25]") # Subtítulo.

        output_analisis = gr.Markdown(label="Resumen Estadístico e Interpretación") # Muestra el resumen estadístico.
        btn_ejecutar_analisis = gr.Button("Ejecutar Análisis Estadístico") # Botón para ejecutar el análisis.

        gr.Markdown("### Visualización de Datos Procesados [25]") # Subtítulo.
        with gr.Row(): # Organiza los componentes en una fila.
            # Dropdown para seleccionar una columna numérica para el gráfico de distribución.
            input_col_distribucion = gr.Dropdown(label="Columna para Gráfico de Distribución (Histograma/Boxplot)", choices=[], interactive=True)

        btn_generar_graficos = gr.Button("Generar Gráficos") # Botón para generar los gráficos.

        with gr.Row(): # Muestra los gráficos generados en una fila.
            plot_correlacion = gr.Plot(label="Mapa de Calor de Correlaciones") # Muestra el mapa de calor.
            plot_distribucion = gr.Plot(label="Distribución y Outliers (Boxplot/Histograma)") # Muestra el gráfico de distribución.
        msg_graficos = gr.Textbox(label="Mensaje de Gráficos") # Muestra mensajes de estado de los gráficos.

    # Define la cuarta pestaña para exportación y reporte.
    with gr.Tab("4. Exportación y Reporte"):
        gr.Markdown("### Exportar Datos Procesados y Generar Log [36]") # Subtítulo.

        radio_formato_exportacion = gr.Radio(
            choices=["CSV", "Excel"],
            label="Seleccionar Formato de Exportación",
            value="CSV"
        )

        btn_generar_archivos = gr.Button("Generar Archivos Finales") # Botón para generar los archivos de salida.
        msg_exportacion = gr.Textbox(label="Resultado de la Exportación") # Muestra el resultado de la exportación.

        output_archivo_datos = gr.File(label="Descargar Datos Procesados") # Componente para descargar el DataFrame procesado.
        output_archivo_log = gr.File(label="Descargar Reporte de Log") # Componente para descargar el reporte de log.

    # Definición de las interacciones entre los componentes de Gradio y las funciones Python.

    # Al hacer clic en 'Cargar y Validar', se ejecuta 'cargar_datos'.
    btn_cargar_datos.click(
        fn=cargar_datos,
        inputs=[input_archivo, radio_separador],
        outputs=[estado_df_gradio, msg_carga_datos, df_vista_previa]
    ).success( # Después de una carga exitosa, actualiza las opciones del dropdown de columnas.
        fn=update_dropdown_choices,
        inputs=[estado_df_gradio],
        outputs=[input_col_distribucion]
    )

    # Al hacer clic en 'Aplicar Limpieza de Nulos', se ejecuta 'manejar_valores_nulos'.
    btn_aplicar_nulos.click(
        fn=manejar_valores_nulos,
        inputs=[estado_df_gradio, input_col_nulos, radio_metodo_nulos],
        outputs=[estado_df_gradio, msg_resultado_nulos, df_vista_previa]
    ).success( # Después de la limpieza, actualiza el dropdown de columnas.
        fn=update_dropdown_choices,
        inputs=[estado_df_gradio],
        outputs=[input_col_distribucion]
    )

    # Al hacer clic en 'Aplicar Normalización / Estandarización', se ejecuta 'aplicar_escalado'.
    btn_aplicar_escalado.click(
        fn=aplicar_escalado,
        inputs=[estado_df_gradio, input_col_escalar, radio_metodo_escalado],
        outputs=[estado_df_gradio, msg_resultado_escalado, df_vista_previa]
    ).success( # Después del escalado, actualiza el dropdown de columnas.
        fn=update_dropdown_choices,
        inputs=[estado_df_gradio],
        outputs=[input_col_distribucion]
    )

    # Al hacer clic en 'Detectar y Tratar Outliers', se ejecuta 'detectar_y_tratar_outliers'.
    btn_detectar_outliers.click(
        fn=detectar_y_tratar_outliers,
        inputs=[estado_df_gradio, input_col_outliers, radio_tratamiento_outliers],
        outputs=[estado_df_gradio, msg_resultado_outliers, df_vista_previa]
    ).success( # Después del tratamiento de outliers, actualiza el dropdown de columnas.
        fn=update_dropdown_choices,
        inputs=[estado_df_gradio],
        outputs=[input_col_distribucion]
    )

    # Al hacer clic en 'Ejecutar Análisis Estadístico', se ejecuta 'ejecutar_analisis'.
    btn_ejecutar_analisis.click(
        fn=ejecutar_analisis,
        inputs=[estado_df_gradio],
        outputs=[output_analisis, gr.State(None)] # El segundo output es un estado que no se muestra, puede ser para la matriz de correlación interna.
    )

    # Al hacer clic en 'Generar Gráficos', se ejecuta 'generar_graficos'.
    btn_generar_graficos.click(
        fn=generar_graficos,
        inputs=[estado_df_gradio, gr.State(None), input_col_distribucion],
        outputs=[plot_correlacion, plot_distribucion, msg_graficos]
    )

    # Al hacer clic en 'Generar Archivos Finales', se ejecuta 'exportar_resultados'.
    btn_generar_archivos.click(
        fn=exportar_resultados,
        inputs=[estado_df_gradio, radio_formato_exportacion],
        outputs=[msg_exportacion, output_archivo_datos, output_archivo_log]
    )

In [28]:
# Inicia la interfaz Gradio si el script se ejecuta directamente.
if __name__ == "__main__": # Asegura que el código se ejecuta solo cuando el script es el programa principal.
    interfaz.launch(inline=True) # Lanza la interfaz de Gradio, incrustándola en la salida de la celda de Colab.

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b582462a301b36f15.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
